In [3]:
import os
import pandas as pd

In [4]:
class RmConf:
    PROJECT_ROOT_DIR = '/Users/gopora/MyStuff/Dev/Workspaces/RiskM'
    INPUT_DIR = os.path.join(PROJECT_ROOT_DIR, "input")
    OUTPUT_DIR = os.path.join(PROJECT_ROOT_DIR, "output")

    PROPHET_INPUT = "pr_input_rm_v01.csv"
    PROPHET_OUTPUT = "pr_output_rm_v01.csv"
    
    PREPARED_TRAINING_DATA = 'training.csv'

    SIM_ID_COL = 'SIMULATION'
    
    TRAIN_SIZE = 0.95
    VAL_SIZE = 0.05
    TEST_SIZE = 0.0


In [5]:
train_data = pd.read_csv(filepath_or_buffer=os.path.join(RmConf.INPUT_DIR, RmConf.PROPHET_INPUT), 
                   sep=';', thousands='.', decimal=',', header=0)
train_data.set_index(['SIMULATION', 'ECONOMY', 'CLASS', 'MEASURE', 'OS_TERM'], inplace=True)
train_data.head()

201712    201801    201802  \
SIMULATION ECONOMY CLASS MEASURE      OS_TERM                                 
1          EUR     VALN  DEF          0        1.000000  1.000371  1.000729   
                         INF          0        1.000000  0.999629  0.999272   
                   CASH  RNY_PC       0       -0.444465 -0.427531 -0.410971   
                   ZCB   SPOT_RATE_PC 0       -0.444465 -0.427531 -0.410971   
                                      1       -0.358000 -0.343001 -0.328141   

                                                 201803    201804    201805  \
SIMULATION ECONOMY CLASS MEASURE      OS_TERM                                 
1          EUR     VALN  DEF          0        1.001072  1.001402  1.001719   
                         INF          0        0.998929  0.998600  0.998284   
                   CASH  RNY_PC       0       -0.394780 -0.378953 -0.363485   
                   ZCB   SPOT_RATE_PC 0       -0.394780 -0.378953 -0.363485   
                                      1       -0.313209 -0.297998 -0.282302   

                                                 201806    201807    201808  \
SIMULATION ECONOMY CLASS MEASURE      OS_TERM                                 
1          EUR     VALN  DEF          0        1.002023  1.002314  1.002594   
                         INF          0        0.997981  0.997691  0.997413   
                   CASH  RNY_PC       0       -0.348373 -0.333610 -0.319193   
                   ZCB   SPOT_RATE_PC 0       -0.348373 -0.333610 -0.319193   
                                      1       -0.265918 -0.248646 -0.230290   

                                                 201809    201810    201811  \
SIMULATION ECONOMY CLASS MEASURE      OS_TERM                                 
1          EUR     VALN  DEF          0        1.002861  1.003116  1.003360   
                         INF          0        0.997147  0.996894  0.996651   
                   CASH  RNY_PC       0       -0.305118 -0.291381 -0.277976   
                   ZCB   SPOT_RATE_PC 0       -0.305118 -0.291381 -0.277976   
                                      1       -0.210655 -0.189546 -0.166775   

                                                 201812  
SIMULATION ECONOMY CLASS MEASURE      OS_TERM            
1          EUR     VALN  DEF          0        1.003593  
                         INF          0        0.996420  
                   CASH  RNY_PC       0       -0.264480  
                   ZCB   SPOT_RATE_PC 0       -0.264480  
                                      1       -0.142154

In [6]:
train_data = pd.DataFrame(train_data.stack())
train_data.columns = ['VALUE']
train_data.index.names = ['SIMULATION', 'ECONOMY', 'CLASS', 'MEASURE', 'OS_TERM', 'MONTH']
train_data.head()

VALUE
SIMULATION ECONOMY CLASS MEASURE OS_TERM MONTH           
1          EUR     VALN  DEF     0       201712  1.000000
                                         201801  1.000371
                                         201802  1.000729
                                         201803  1.001072
                                         201804  1.001402

In [7]:
train_data.reset_index(inplace=True)
train_data['EC_CL_MS_OS'] = train_data['ECONOMY'] + '_' + train_data['CLASS'] + '_' + train_data['MEASURE'] + '_' + train_data.OS_TERM.map(str)
train_data.drop(columns=['ECONOMY', 'CLASS', 'MEASURE', 'OS_TERM'], inplace=True)
train_data.set_index(['SIMULATION', 'EC_CL_MS_OS', 'MONTH'], inplace=True)
train_data.head()

VALUE
SIMULATION EC_CL_MS_OS    MONTH           
1          EUR_VALN_DEF_0 201712  1.000000
                          201801  1.000371
                          201802  1.000729
                          201803  1.001072
                          201804  1.001402

In [8]:
train_data = train_data.unstack(1)
train_data.columns = train_data.columns.droplevel()
train_data.sort_index(inplace=True)
train_data.head()

EC_CL_MS_OS        EUR_CASH_RNY_PC_0  EUR_EQUITY_RET_PC_0  \
SIMULATION MONTH                                            
1          201712          -0.444465             0.000000   
           201801          -0.427531            -0.037114   
           201802          -0.410971            -0.035698   
           201803          -0.394780            -0.034312   
           201804          -0.378953            -0.032958   

EC_CL_MS_OS        EUR_EQUITY_RNY_PC_0  EUR_EQU_DTFS_RET_PC_0  \
SIMULATION MONTH                                                
1          201712             2.890000               0.000000   
           201801             2.882558              -0.037114   
           201802             2.875292              -0.035698   
           201803             2.868199              -0.034312   
           201804             2.861274              -0.032958   

EC_CL_MS_OS        EUR_EQU_DTFS_RNY_PC_0  EUR_EQU_DTFS_VOL_PC_1  \
SIMULATION MONTH                                                  
1          201712                    0.5                    5.0   
           201801                    0.5                    5.0   
           201802                    0.5                    5.0   
           201803                    0.5                    5.0   
           201804                    0.5                    5.0   

EC_CL_MS_OS        EUR_EQU_F_RET_PC_0  EUR_EQU_F_RNY_PC_0  \
SIMULATION MONTH                                            
1          201712                 0.0                 0.0   
           201801                 0.0                 0.0   
           201802                 0.0                 0.0   
           201803                 0.0                 0.0   
           201804                 0.0                 0.0   

EC_CL_MS_OS        EUR_FIXED_PAR_YLD_PC_0  EUR_FIXED_PAR_YLD_PC_1  \
SIMULATION MONTH                                                    
1          201712               -0.444465               -0.358000   
           201801               -0.427531               -0.343001   
           201802               -0.410971               -0.328141   
           201803               -0.394780               -0.313209   
           201804               -0.378953               -0.297998   

EC_CL_MS_OS                 ...            EUR_ZCB_SPOT_RATE_PC_28  \
SIMULATION MONTH            ...                                      
1          201712           ...                           1.837040   
           201801           ...                           1.849010   
           201802           ...                           1.860951   
           201803           ...                           1.872862   
           201804           ...                           1.884746   

EC_CL_MS_OS        EUR_ZCB_SPOT_RATE_PC_29  EUR_ZCB_SPOT_RATE_PC_3  \
SIMULATION MONTH                                                     
1          201712                 1.897426               -0.088375   
           201801                 1.909213               -0.064254   
           201802                 1.920969               -0.039920   
           201803                 1.932694               -0.015389   
           201804                 1.944390                0.009325   

EC_CL_MS_OS        EUR_ZCB_SPOT_RATE_PC_30  EUR_ZCB_SPOT_RATE_PC_4  \
SIMULATION MONTH                                                     
1          201712                 1.956245                0.069166   
           201801                 1.967833                0.092209   
           201802                 1.979389                0.115238   
           201803                 1.990913                0.138285   
           201804                 2.002407                0.161379   

EC_CL_MS_OS        EUR_ZCB_SPOT_RATE_PC_5  EUR_ZCB_SPOT_RATE_PC_6  \
SIMULATION MONTH                                                    
1          201712                0.209026                0.346886   
           201801                0.231846             

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13013 entries, (1, 201712) to (1001, 201812)
Data columns (total 78 columns):
EUR_CASH_RNY_PC_0          13013 non-null float64
EUR_EQUITY_RET_PC_0        13013 non-null float64
EUR_EQUITY_RNY_PC_0        13013 non-null float64
EUR_EQU_DTFS_RET_PC_0      13013 non-null float64
EUR_EQU_DTFS_RNY_PC_0      13013 non-null float64
EUR_EQU_DTFS_VOL_PC_1      13013 non-null float64
EUR_EQU_F_RET_PC_0         13013 non-null float64
EUR_EQU_F_RNY_PC_0         13013 non-null float64
EUR_FIXED_PAR_YLD_PC_0     13013 non-null float64
EUR_FIXED_PAR_YLD_PC_1     13013 non-null float64
EUR_FIXED_PAR_YLD_PC_10    13013 non-null float64
EUR_FIXED_PAR_YLD_PC_11    13013 non-null float64
EUR_FIXED_PAR_YLD_PC_12    13013 non-null float64
EUR_FIXED_PAR_YLD_PC_13    13013 non-null float64
EUR_FIXED_PAR_YLD_PC_14    13013 non-null float64
EUR_FIXED_PAR_YLD_PC_15    13013 non-null float64
EUR_FIXED_PAR_YLD_PC_16    13013 non-null float64
EUR_FIXED_PAR_YLD_PC_1

In [5]:
train_targets = pd.read_csv(filepath_or_buffer=os.path.join(RmConf.INPUT_DIR, RmConf.PROPHET_OUTPUT), 
                   sep=';', thousands='.', decimal=',', header=0)
train_targets.set_index('SIMULATION', inplace=True)
train_targets.head()

,OWN_FUNDS
SIMULATION,
1,7.786157e+08
2,7.797727e+08
3,7.791493e+08
4,7.785029e+08
5,7.829051e+08


In [6]:
train_targets.describe()

,OWN_FUNDS
count,1.001000e+03
mean,7.799244e+08
std,2.688279e+06
min,7.695445e+08
25%,7.781101e+08
50%,7.797519e+08
75%,7.814491e+08
max,7.953346e+08


In [13]:
import numpy as np
from math import sqrt
mean = np.mean(train_targets)

sqrt(np.sum((train_targets - mean)**2))

85010837.07243063

In [84]:
from sklearn.model_selection import ShuffleSplit

def split_data(x, y, train_size, val_size, test_size):
    if val_size + test_size > 0:
        x_tr,y_tr, x_v, y_v = split_train_test(x, y, train_size, val_size + test_size)

        if val_size == 0:
            x_te = x_v
            y_te = y_v
            x_v = None
            y_v = None
        elif test_size == 0:
            x_te = None
            y_te = None
        else:
            x_v, y_v, x_te, y_te = split_train_test(x_v, y_v,
                                                    val_size / (val_size + test_size),
                                                    test_size / (val_size + test_size))
    else:
        x_tr = x
        y_tr = y
        x_v = None
        y_v = None
        x_te = None
        y_te = None

    return x_tr, y_tr, x_v, y_v, x_te, y_te


def split_train_test(x, y, train_size, test_size):
    split = ShuffleSplit(n_splits=1, train_size=train_size, test_size=test_size, random_state=None)
    res = split.split(y)

    x_tr = None
    y_tr = None
    x_te = None
    y_te = None

    for train_i, test_i in res:
        x_tr = pd.DataFrame(x.loc[x.index.levels[0][train_i].values])
        y_tr = pd.DataFrame(y.iloc[train_i])
        x_te = pd.DataFrame(x.loc[x.index.levels[0][test_i].values])
        y_te = pd.DataFrame(y.iloc[test_i])

    x_tr.reset_index(inplace=True)
    x_tr.set_index(['SIMULATION', 'MONTH'], inplace=True)
    x_tr.sort_index(inplace=True)
    print(x_tr.index.levels[0])
    print(x_tr.shape)
    y_tr.sort_index(inplace=True)
    print(y_tr.shape)
    x_te.reset_index(inplace=True)
    x_te.set_index(['SIMULATION', 'MONTH'], inplace=True)
    x_te.sort_index(inplace=True)
    print(x_te.shape)
    y_te.sort_index(inplace=True)
    print(y_te.shape)

    return x_tr, y_tr, x_te, y_te


In [85]:
train_x = train_data
train_y = train_targets
train_x, train_y, val_x, val_y, test_x, test_y = split_data(x=train_x, y=train_y,
                                                            train_size=0.95,
                                                            val_size=0.04,
                                                            test_size=0.01)

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   11,
            ...
             992,  993,  994,  995,  996,  997,  998,  999, 1000, 1001],
           dtype='int64', name='SIMULATION', length=950)
(12350, 78)
(950, 1)
(663, 78)
(51, 1)
Int64Index([ 12,  22,  26,  47,  59,  69, 122, 158, 160, 182, 187, 231, 245,
            392, 416, 431, 433, 482, 572, 574, 575, 598, 613, 623, 630, 635,
            640, 672, 682, 683, 697, 707, 724, 794, 803, 815, 832, 863, 943,
            965],
           dtype='int64', name='SIMULATION')
(520, 78)
(40, 1)
(143, 78)
(11, 1)


In [31]:
train_y

,OWN_FUNDS
SIMULATION,
1,7.786157e+08
2,7.797727e+08
3,7.791493e+08
4,7.785029e+08
5,7.829051e+08
6,7.807969e+08
7,7.780554e+08
8,7.779813e+08
9,7.862044e+08


In [32]:
val_y

,OWN_FUNDS
SIMULATION,
81,7.780135e+08
95,7.797655e+08
96,7.777020e+08
103,7.833738e+08
125,7.820089e+08
129,7.792739e+08
136,7.825438e+08
140,7.851063e+08
141,7.795906e+08


In [33]:
val_x

EC_CL_MS_OS        EUR_CASH_RNY_PC_0  EUR_EQUITY_RET_PC_0  \
SIMULATION MONTH                                            
81         201712          -0.444500             0.000000   
           201801          -0.401300            -9.441200   
           201802          -0.425200             6.544698   
           201803          -0.387600            -0.097009   
           201804          -0.282500            -5.458856   
           201805          -0.248000            -2.115547   
           201806          -0.318300            -3.335680   
           201807          -0.289400            -3.811001   
           201808          -0.330800             4.350945   
           201809          -0.384700             7.671004   
           201810          -0.391300             6.724971   
           201811          -0.384300            -3.115467   
           201812          -0.316900            -2.038775   
95         201712          -0.444500             0.000000   
           201801          -0.431100            -0.946700   
           201802          -0.361200             0.333457   
           201803          -0.381300             1.399124   
           201804          -0.379800            -0.765177   
           201805          -0.266999             2.214334   
           201806          -0.259799             8.101263   
           201807          -0.255399            -4.464232   
           201808          -0.162299            -6.550726   
           201809          -0.110999             0.773542   
           201810          -0.105399             5.518181   
           201811          -0.139399            10.116207   
           201812          -0.188299             3.490842   
96         201712          -0.444500             0.000000   
           201801          -0.420900            -6.067900   
           201802          -0.425400            -5.535807   
           201803          -0.516100             1.549494   
...                              ...                  ...   
933        201809          -0.353400             4.748274   
           201810          -0.383400             0.422702   
           201811          -0.352400            -3.596247   
           201812          -0.350400             1.534817   
950        201712          -0.444500             0.000000   
           201801          -0.473100             0.602300   
           201802          -0.524300            -4.165710   
           201803          -0.509900             0.029353   
           201804          -0.539300             2.723979   
           201805          -0.527600             3.228831   
           201806          -0.554500             1.719642   
           201807          -0.550900            -3.728847   
           201808          -0.525500            -0.254031   
           201809          -0.462500             3.727771   
           201810          -0.479400             3.908912   
           201811          -0.446000            -2.366607   
           201812          -0.458700            -3.779133   
994        201712          -0.444500             0.000000   
           201801          -0.421300            -2.698400   
           201802          -0.422800            -1.676334   
           201803          -0.450700            -4.220737   
           201804          -0.436800            -1.279677   
           201805          -0.429600            -2.257575   
           201806          -0.394500             2.909259   
           201807          -0.327899            -7.717991   
           201808          -0.330299            -2.080808   
           201809          -0.293599             1.310748   
           201810          -0.257799             1.485632   
           201811          -0.173399            -8.812166   
           201812          -0.109899             0.886672   

EC_CL_MS_OS        EUR_EQUITY_RNY_PC_0  EUR_EQU_DTFS_RET_PC_0  \
SIMULATION MONTH                                                
81         201712         

In [29]:
test_x

,EC_CL_MS_OS,EUR_CASH_RNY_PC_0,EUR_EQUITY_RET_PC_0,EUR_EQUITY_RNY_PC_0,EUR_EQU_DTFS_RET_PC_0,EUR_EQU_DTFS_RNY_PC_0,EUR_EQU_DTFS_VOL_PC_1,EUR_EQU_F_RET_PC_0,EUR_EQU_F_RNY_PC_0,EUR_FIXED_PAR_YLD_PC_0,EUR_FIXED_PAR_YLD_PC_1,...,EUR_ZCB_SPOT_RATE_PC_28,EUR_ZCB_SPOT_RATE_PC_29,EUR_ZCB_SPOT_RATE_PC_3,EUR_ZCB_SPOT_RATE_PC_30,EUR_ZCB_SPOT_RATE_PC_4,EUR_ZCB_SPOT_RATE_PC_5,EUR_ZCB_SPOT_RATE_PC_6,EUR_ZCB_SPOT_RATE_PC_7,EUR_ZCB_SPOT_RATE_PC_8,EUR_ZCB_SPOT_RATE_PC_9
SIMULATION,MONTH,,,,,,,,,,,,,,,,,,,,,


In [8]:
X = train_data.as_matrix()
y = train_targets.as_matrix()

X = X.reshape(-1,13,78)

X.shape

(1001, 13, 78)

In [9]:
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint
from keras.layers import Input, LSTM, RepeatVector, Dense, SimpleRNN, TimeDistributed, Concatenate, CuDNNGRU, GRU
from keras.models import Model, load_model

inputs = Input(shape=(13, 78), name='Input_Sequence')
print(inputs.shape)
enc_output, enc_state_h_1 = \
    SimpleRNN(units=200, activation='relu', return_state=True, return_sequences=True, name='Encoder_RNN_1')(inputs)
print(enc_output.shape, enc_state_h_1.shape)
enc_output, enc_state_h_2 = \
    SimpleRNN(units=200, activation='relu', return_state=True, return_sequences=True, name='Encoder_RNN_2')(enc_output)
print(enc_output.shape, enc_state_h_2.shape)
enc_output, enc_state_h_3 = \
    SimpleRNN(units=200, activation='relu', return_state=True, name='Encoder_RNN_3')(enc_output)
print(enc_output.shape, enc_state_h_2.shape)
encoded = Concatenate(axis=1, name='Encoder_State_Fusion')([enc_state_h_1, enc_state_h_2, enc_state_h_3])
print(encoded.shape)

encoded = Dense(100, name='Latent')(encoded)
print(encoded.shape)
decoded = Dense(1, name='Prediction')(encoded)
print(decoded.shape)


model = Model(inputs, decoded)

model.compile(optimizer='adam', loss='mse', metrics=None)

/Users/gopora/MyStuff/Dev/python-virtual-envs/pie36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(?, 13, 78)
(?, ?, 200) (?, 200)
(?, ?, 200) (?, 200)
(?, 200) (?, 200)
(?, 600)
(?, 100)
(?, 1)


In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Sequence (InputLayer)     (None, 13, 78)       0                                            
__________________________________________________________________________________________________
Encoder_RNN_1 (SimpleRNN)       [(None, 13, 200), (N 55800       Input_Sequence[0][0]             
__________________________________________________________________________________________________
Encoder_RNN_2 (SimpleRNN)       [(None, 13, 200), (N 80200       Encoder_RNN_1[0][0]              
__________________________________________________________________________________________________
Encoder_RNN_3 (SimpleRNN)       [(None, 200), (None, 80200       Encoder_RNN_2[0][0]              
__________________________________________________________________________________________________
Encoder_St

In [11]:
history = model.fit(
        x=[X], y=y,
        batch_size=32,
        epochs=1000,
        verbose=1,
        callbacks=None,
        shuffle=True,
        initial_epoch=0,
        steps_per_epoch=None,
        validation_data=None)

Epoch 1/1000
1001/1001 [==============================] - 2s 2ms/step - loss: 461593785539577984.0000
Epoch 2/1000
1001/1001 [==============================] - 1s 600us/step - loss: 41489398036931048.0000
Epoch 3/1000
1001/1001 [==============================] - 1s 596us/step - loss: 1761303353670976.2500
Epoch 4/1000
1001/1001 [==============================] - 1s 595us/step - loss: 773583609557650.7500
Epoch 5/1000
1001/1001 [==============================] - 1s 596us/step - loss: 737407259381854.1250
Epoch 6/1000
 128/1001 [==>...........................] - ETA: 0s - loss: 634873307987968.0000

KeyboardInterrupt: 